# Tika Instructions
- DONE - first, git clone http://github.com/chrismattmann/tika-similarity.git and then git clone http://github.com/chrismattmann/etllib.git You will use ETLlib for tsv2json

- then cd tika-similarity and then inside there, do pip install -r requirements.txt . This will take care of installing Tika Similarity's dependencies. Then inside of etllib do python setup.py install that will install the associated scripts like tsv2json etc (

- make sure that you already have your dataset by this point, with all of your additional features added

- then run tsv2json and generate approx 95k JSON files in a directory 

- once the files are generated in there, you can run similarity.py on the directory with the 95k JSON files

- it will generate a file called similarity-scores.txt that has all the resemblance and pairwise similarity scores for each item in your posts with the additional features

- then you should run cluster-json.py that will read the similarity-scores.txt and from there use the associated threshold to perform hierarchical agglomerative clustering and to generate clusters.json once you have that file generated you can browse the clusters

- you can mess around with different metrics, e.g., try the edit distance version of similarity.py or use value versus key based similarity and so on